In [1]:
from pathlib import Path
import copy
from tqdm import tqdm 

import torch
import pyscf
import numpy as np
import opt_einsum as oe

from cadft.utils import Grid
from cadft import CC_DFT_DATA, Mol


/home/dhem/anaconda3/envs/pyscf/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
molecular = copy.deepcopy(Mol["Pentane"])
molecular[0][1] += 0.12345

dft2cc = CC_DFT_DATA(
    molecular,
    name="test",
    basis="cc-pvdz",
    if_basis_str=True,
)

mf = pyscf.scf.RHF(dft2cc.mol)
mf.kernel()
mycc = pyscf.cc.CCSD(mf)
mycc.kernel()
dm1_cc = mycc.make_rdm1(ao_repr=True)
dm2_cc = mycc.make_rdm2(ao_repr=True)
e_cc = mycc.e_tot

mdft = pyscf.scf.RKS(dft2cc.mol)
mdft.xc = "b3lyp"
mdft.kernel()
e_dft = mdft.e_tot
dm1_dft = mdft.make_rdm1(ao_repr=True)

grids = Grid(dft2cc.mol)
coords = grids.coords
weights = grids.weights
ao_2 = pyscf.dft.numint.eval_ao(dft2cc.mol, coords, deriv=2)
ao_0 = ao_2[0, :, :]
ao_value = ao_2[:4, :, :]
ao_2_diag = ao_2[4, :, :] + ao_2[7, :, :] + ao_2[9, :, :]

rho_dft = pyscf.dft.numint.eval_rho(dft2cc.mol, ao_value, dm1_dft, xctype="GGA")
rho_cc = pyscf.dft.numint.eval_rho(dft2cc.mol, ao_value, dm1_cc, xctype="GGA")

rho_dft_grids = grids.vector_to_matrix(rho_dft[0])
rho_cc_grids = grids.vector_to_matrix(rho_cc[0])

In [3]:
exc_over_dm_cc_grids = np.zeros_like(rho_dft[0])
exc_over_dm_cc_grids = -pyscf.dft.libxc.eval_xc("b3lyp", rho_dft)[0] * rho_dft[0]
expr_rinv_dm2_r = oe.contract_expression(
    "ijkl,i,j,kl->",
    0.5 * (dm2_cc - oe.contract("pq,rs->pqrs", dm1_dft, dm1_dft))
    + 0.05 * oe.contract("pr,qs->pqrs", dm1_dft, dm1_dft),
    (dft2cc.mol.nao,),
    (dft2cc.mol.nao,),
    (dft2cc.mol.nao, dft2cc.mol.nao),
    constants=[0],
    optimize="optimal",
)

for i, coord in enumerate(tqdm(coords)):
    ao_0_i = ao_value[0][i]
    if np.linalg.norm(ao_0_i) < 1e-10:
        continue
    with dft2cc.mol.with_rinv_origin(coord):
        rinv = dft2cc.mol.intor("int1e_rinv")
        exc_over_dm_cc_grids[i] += expr_rinv_dm2_r(ao_0_i, ao_0_i, rinv)

    for i_atom in range(dft2cc.mol.natm):
        exc_over_dm_cc_grids[i] -= (
            (rho_cc[0][i] - rho_dft[0][i])
            * dft2cc.mol.atom_charges()[i_atom]
            / np.linalg.norm(dft2cc.mol.atom_coords()[i_atom] - coord)
        )

dm1_cc_mo = mycc.make_rdm1(ao_repr=False)
eigs_e_dm1, eigs_v_dm1 = np.linalg.eigh(dm1_cc_mo)
eigs_v_dm1 = mf.mo_coeff @ eigs_v_dm1
for i in range(np.shape(eigs_v_dm1)[1]):
    part = oe.contract(
        "pm,m,n,pn->p",
        ao_0,
        eigs_v_dm1[:, i],
        eigs_v_dm1[:, i],
        ao_2_diag,
    )
    exc_over_dm_cc_grids += -part * eigs_e_dm1[i] / 2

for i in range(dft2cc.mol.nelec[0]):
    part = oe.contract(
        "pm,m,n,pn->p",
        ao_0,
        mdft.mo_coeff[:, i],
        mdft.mo_coeff[:, i],
        ao_2_diag,
    )
    exc_over_dm_cc_grids += part

error = np.sum(exc_over_dm_cc_grids * weights) - (e_cc - e_dft)

print(f"Error: {(1e3 * error):.5f} mHa")

  0%|          | 0/45300 [00:00<?, ?it/s]

100%|██████████| 45300/45300 [00:20<00:00, 2217.57it/s]

Error: 0.00001 mHa


In [4]:
ene_cc_grids = grids.vector_to_matrix(exc_over_dm_cc_grids)
rho_cc_grids_pred = np.zeros_like(ene_cc_grids)
ene_cc_grids_pred = np.zeros_like(ene_cc_grids)

In [5]:
from cadft.utils import ModelDict

Modeldict = ModelDict(200, 3, 1, "cuda")
Modeldict.load_model("2024-05-31-11-36-26")

Model loaded from checkpoints/checkpoint-ccdft-2024-05-31-11-36-26-200/1-2500.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-31-11-36-26-200/2-2500.pth


In [6]:
ene_ = 0
for i_atom in range(rho_dft_grids.shape[0]):
    for i in range(rho_dft_grids.shape[1]):
        input_ = rho_dft_grids[i_atom, i, :]
        input_gpu = torch.tensor(input_, dtype=torch.float64).to("cuda")
        output_ = Modeldict.model_dict["1"](input_gpu).detach().cpu().numpy()
        rho_cc_grids_pred[i_atom, i, :] = output_ / 1000 + input_
        output_ = Modeldict.model_dict["2"](input_gpu).detach().cpu().numpy()
        ene_cc_grids_pred[i_atom, i, :] = output_ / 1000
        ene_ += np.sum(output_)
ene_ - (e_cc - e_dft) * 1000

46.93391122537693

In [7]:
print(np.sum(np.abs(grids.matrix_to_vector(rho_cc_grids_pred - rho_cc_grids) * weights)))
print(np.sum(np.abs(grids.matrix_to_vector(rho_dft_grids - rho_cc_grids) * weights)))

0.03085302682074169
0.01546141519926766


In [11]:
print(
    np.max(
        np.abs(
            grids.matrix_to_vector(ene_cc_grids_pred)
            - grids.matrix_to_vector(ene_cc_grids) * weights
        )
    )
)

print(np.max(np.abs(grids.matrix_to_vector(ene_cc_grids) * weights)))

print(
    np.sum(
        grids.matrix_to_vector(ene_cc_grids_pred)
        - grids.matrix_to_vector(ene_cc_grids) * weights
    )
)

7.512282984314143e-05
5.891937093377168e-06
0.04693390471250075
